In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pytesseract as tess
import time
import xlsxwriter
import openpyxl
import xlrd
from PIL import Image
import os
companyid=[]
file_name="state.xlsx"
wb = xlrd.open_workbook(file_name)
sheet = wb.sheet_by_index(0)
for i in range(1,sheet.nrows):
    companyid.append(sheet.cell_value(i, 0))
if os.path.exists(file_name):
    os.remove(file_name)
location=file_name.split('.')[0]+'_Charges'
if os.path.exists(location):
    pass
else:
    os.mkdir(location)
outworkbook=xlsxwriter.Workbook(file_name)
bold=outworkbook.add_format({'bold': True})
company_sheet=outworkbook.add_worksheet('Company')
resulttab1=['CIN','Company Name','ROC Code','Registration Number','Company Category','Company SubCategory','Class of Company','Authorised Capital(Rs)','Paid up Capital(Rs)','Number of Members(Applicable in case of company without Share Capital)','Date of Incorporation','Registered Address','Address other than R/o where all or any books of account and papers are maintained','Email Id','Whether Listed or not','ACTIVE compliance','Suspended at stock exchange','Date of last AGM','Date of Balance Sheet','Company Status(for efiling)','Charges','DIN/PAN']
column=0
for data in resulttab1:
    company_sheet.write(0,column,data,bold)
    column=column+1
llp_sheet=outworkbook.add_worksheet('LLP')
resulttab3=['LLPIN','LLP Name','Number of Partners','Number of Designated Partners','ROC Code','Date of Incorporation','Registered Address','Email Id','Previous firm/ company details, if applicable','Total Obligation of Contribution','Main division of business activity to be carried out in India','Description of main division','Date of last financial year end date for which Statement of Accounts and Solvency filed','Date of last financial year end date for which Annual Return filed','LLP Status','Charges','DIN/PAN']
column=0
for data in resulttab3:
    llp_sheet.write(0,column,data,bold)
    column=column+1
fcrn_sheet=outworkbook.add_worksheet('FCRN')
resulttab2=['FCRN','Company Name','Date of Incorporation','Country of Incorporation','Registered Address','Email Id','Foreign Company with Share Capital','Company Status','Type of Office','Details','Main division of business activity to be carried out in India','Description of main division','Charges','DIN/PAN']
column=0
for data in resulttab2:
    fcrn_sheet.write(0,column,data,bold)
    column=column+1
na_sheet=outworkbook.add_worksheet('NA')
na_sheet.write(0,0,'Leftover',bold)
outworkbook.close()
for data in companyid:
    result_length=0
    result=''
    sheet=100
    final=[]
    length=len(data)
    if length==21:
        result='resultTab1'
        result_length=len(resulttab1)
        sheet=0
    elif length==6:
        result='resultTab2'
        result_length=len(resulttab2)
        sheet=2
    elif length==8:
        result='resultTab3'
        result_length=len(resulttab3)
        sheet=1       
    else:
        sheet=3
        wb =openpyxl.load_workbook(file_name)
        ws = wb.worksheets[sheet]
        final.append(data)
        ws.append(final)
        wb.save(file_name)
        continue
    while(1):
        try:
            driver=webdriver.Chrome(executable_path=r'chromedriver.exe')
            driver.get('http://www.mca.gov.in/mcafoportal/viewCompanyMasterData.do')
            old_url = driver.current_url
            time.sleep(5)
            with open('filename.png', 'wb') as file:
                file.write(driver.find_element_by_id('captcha').screenshot_as_png)
            tess.pytesseract.tesseract_cmd=r'Tesseract-OCR\tesseract.exe'
            img=Image.open('filename.png')
            capcha_input=tess.image_to_string(img)
            cin=driver.find_element_by_xpath('//*[@id="companyID"]')
            cin.send_keys(data)
            capcha= driver.find_element_by_id('userEnteredCaptcha')
            capcha_final=capcha_input.split('\n')[2]
            capcha_final=capcha_final.lower()
            capcha.send_keys(capcha_final)
            driver.find_element_by_id('companyLLPMasterData_0').click()
            time.sleep(2)
            if driver.find_element_by_id(result):
                html = driver.page_source
                soup=BeautifulSoup(html,'html.parser')
                for data_value in soup.find_all('tbody')[0].find_all('tr'):
                    table_1=data_value.find_all('td')
                    final.append(table_1[1].text.strip())
                if (soup.find_all('tbody')[1].find_all('tr')[1].text.strip()=='No Charges Exists for Company/LLP'):
                    final.append('No')
                else:
                    final.append('Yes')
                    charge_location=location+'/'+data+'.xlsx'
                    charge_book=xlsxwriter.Workbook(charge_location)
                    charge_sheet=charge_book.add_worksheet('Charges')
                    charge_row=0
                    charge_column=0
                    for value in soup.find_all('tbody')[1].find_all('tr')[0].find_all('th'):
                        charge_sheet.write(charge_row,charge_column,value.text.strip(),bold)
                        charge_column=charge_column+1
                    charge_column=0    
                    charge_row=charge_row+1
                    counter=1
                    for value in soup.find_all('tbody')[1].find_all('td'):
                        if (counter%5==0):
                            charge_sheet.write(charge_row,charge_column,value.text.strip())
                            charge_column=0
                            charge_row=charge_row+1
                            counter=counter+1
                            continue
                        charge_sheet.write(charge_row,charge_column,value.text.strip())
                        charge_column=charge_column+1
                        counter=counter+1
                    charge_book.close()    
                din=soup.find_all('tbody')[2].find_all('tr')    
                din_length=len(din)
                combined_din=''
                if din_length<2:
                    combined_din=combined_din+'NA'
                else:
                    for i in range(1,din_length):
                        combined_din=combined_din+(din[i].find_all('td')[0].text.strip())+','
                    combined_din=combined_din[:-1]
                    final.append(combined_din)
                wb =openpyxl.load_workbook(file_name)
                ws = wb.worksheets[sheet]
                ws.append(final)
                wb.save(file_name)
                driver.close()
                break
        except:
            html1 = driver.page_source
            soup1=BeautifulSoup(html1,'html.parser')
            error_length=len(soup1.find_all(class_='errorMessage'))
            if error_length>0:
                error_message=soup1.find_all(class_='errorMessage')[0].text.strip()
                if (error_message=='Entered CIN/LLPIN/FLLPIN/FCRN is not found. Please enter valid details.'):
                    final=[]
                    final.append(data)
                    for i in range(1,result_length):
                        final.append('NA')
                    wb =openpyxl.load_workbook(file_name)
                    ws = wb.worksheets[sheet]
                    ws.append(final)
                    wb.save(file_name)
                    driver.close()
                    break
                else:
                    driver.close()
                    continue
            else:
                driver.close()
                continue 
print('Its done')                

Its done
